<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/syntheticPoseImageGeneration/tests/Pose_with_Style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pose with Style: Detail-Preserving Pose-Guided Image Synthesis with Conditional StyleGAN

[GitHub](https://github.com/BadourAlBahar/pose-with-style)

## 1. Preparations

Before start, make sure that you choose

Runtime Type = Python 3
Hardware Accelerator = GPU

In [1]:
!nvidia-smi

Mon Jul 18 08:51:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. linking nextcloud

Connecting to the external NextCloud drive 

In [2]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

input_folder, output_folder = create_io(database=nextcloud,topic='syntheticPoseImageGeneration',library='PoseStyle')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2510  100  2510    0     0  13141      0 --:--:-- --:--:-- --:--:-- 13141
what's the username for nextcloud? colab
what's the password for user colab? ··········
0
Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  


## 3. clone GIT repo

In [3]:
import os
root_path = '/content/PoseStyle'

# clone the repository
if not os.path.exists(root_path):
  !git clone https://github.com/BadourAlBahar/pose-with-style {root_path}

%ls

Cloning into '/content/PoseStyle'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 43 (delta 0), reused 0 (delta 0), pack-reused 38
Unpacking objects: 100% (43/43), done.
database/  database_mod.py  PoseStyle/  __pycache__/  sample_data/
